In [1]:
!pip install emp-orderly emp-orderly-types eth-typeshed-py pandas-ta numpy==1.26.4 matplotlib setuptools

In [2]:
from dotenv import load_dotenv
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, SMA,
    EmpyrealOrderlySDK,
)
from emp_orderly_types import PerpetualAssetType, Interval

load_dotenv()

/home/ademola/work/stackup/bounties/ajibola/orderly-strategy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [4]:
class ExtendedSmaCross(Strategy):
    n1: int = 10
    n2: int = 30

    @classmethod
    def update_lags(cls, n1, n2):
        cls.n1 = n1
        cls.n2 = n2

    def init(self):
        close = self.data.close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        # probability, sentiment = self.get_sentiment()
        if (
            crossover(self.sma1, self.sma2)
        ):
            self.position.close()
            self.buy(size=.999)
        elif (
            crossover(self.sma2, self.sma1)
        ):
            self.position.close()
            self.sell(size=.999)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(ExtendedSmaCross)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.DOGE,
)

# backtest
tester.backtest()

Start                     2024-09-26 00:15:00
End                       2024-10-02 23:10:00
Duration                      6 days 22:55:00
Exposure Time [%]                   78.842315
Equity Final [$]                  1264.432562
Equity Peak [$]                   1285.205783
Return [%]                          26.443256
Buy & Hold Return [%]               -0.382629
Return (Ann.) [%]             20565028.728445
Volatility (Ann.) [%]          11813174.82821
Sharpe Ratio                         1.740855
Sortino Ratio                             inf
Calmar Ratio                   4036863.793866
Max. Drawdown [%]                   -5.094308
Avg. Drawdown [%]                   -1.122615
Max. Drawdown Duration        1 days 11:55:00
Avg. Drawdown Duration        0 days 03:03:00
# Trades                                   52
Win Rate [%]                        48.076923
Best Trade [%]                       9.074135
Worst Trade [%]                     -1.657663
Avg. Trade [%]                    

In [5]:
# plot
tester.plot(show_price_data=False)
plt.show()

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ademola/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.
